In [89]:
from pymor.analyticalproblems.elliptic import EllipticProblem
from pymor.domaindescriptions.basic import RectDomain
from pymor.functions.basic import *
from pymor.functions.bitmap import *
from pymor.parameters.spaces import CubicParameterSpace
from pymor.discretizers.elliptic import discretize_elliptic_cg

domain = RectDomain()

rhs = ExpressionFunction('(( (x[...,0]-0.25)**2 + (x[...,1]-0.5)**2) <= 0.001) * 1.', 2, ())

parameter_range=(0., 100.)
parameter_space = CubicParameterSpace({'eps': (), 'k': (), 'N': ()}, *parameter_range)

def func(x, mu):
    return 1. - (np.sqrt( (np.mod(x[...,0],1./mu["N"])-0.5/mu["N"])**2 + (np.mod(x[...,1],1./mu["N"])-0.5/mu["N"])**2) <= 0.3/mu["N"]) * (1.-mu["eps"])
diffusion = GenericFunction(func, dim_domain =2, parameter_type=parameter_space.parameter_type)

#diffusion = ExpressionFunction( '1. - (sqrt( (np.mod(x[...,0],1./K)-0.5/K)**2 + (np.mod(x[...,1],1./K)-0.5/K)**2) <= 0.3/K) * 0.999', 2, (), values={'K': 10})

diffusion = BitmapFunction('coeff.bmp', range=[1e-5, 1.])

p = EllipticProblem(
    diffusion_functions=[diffusion],
    diffusion_functionals=[1.],
    reaction_functions=[ConstantFunction(1., dim_domain=domain.dim)],
    reaction_functionals=[ExpressionParameterFunctional('-k**2', parameter_type=parameter_space.parameter_type)],
    domain=domain,
    rhs = rhs,
    parameter_space=parameter_space
)

In [90]:
parameter_space.parameter_type

ParameterType({N: (), eps: (), k: ()})

In [91]:
d,data = discretize_elliptic_cg(p, diameter = 1./100)

In [92]:
u = d.solve({'eps': 1e-3, 'k': 28, 'N': 4})

36:52 StationaryDiscretization: Solving EllipticProblem_CG for {N: 4, eps: 0.001, k: 28} ...
36:52 DiffusionOperatorP1: Calulate gradients of shape functions transformed by reference map ...
36:52 DiffusionOperatorP1: Calculate all local scalar products beween gradients ...
36:52 DiffusionOperatorP1: Determine global dofs ...
36:52 DiffusionOperatorP1: Boundary treatment ...
36:52 DiffusionOperatorP1: Assemble system matrix ...
36:52 DiffusionOperatorP1: Calulate gradients of shape functions transformed by reference map ...
36:52 DiffusionOperatorP1: Calculate all local scalar products beween gradients ...
36:52 DiffusionOperatorP1: Determine global dofs ...
36:52 DiffusionOperatorP1: Boundary treatment ...
36:52 DiffusionOperatorP1: Assemble system matrix ...
36:52 L2ProductP1: Integrate the products of the shape functions on each element
36:52 L2ProductP1: Determine global dofs ...
36:52 L2ProductP1: Boundary treatment ...
36:52 L2ProductP1: Assemble system matrix ...


In [93]:
d.visualize(u)